# Sai Ram

https://www.kaggle.com/c/santander-customer-transaction-prediction

In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import seaborn as sns
from sklearn import metrics, preprocessing
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold, train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


In [3]:
random_state = 42
np.random.seed(random_state)

In [4]:
# read data

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

test_ID = df_test['ID_code'].values
Y = df_train.target.values.astype(np.float32)

df_train = df_train.drop(['ID_code','target'], axis=1)
df_test = df_test.drop(['ID_code'], axis=1)

X = pd.concat([df_train,df_test], axis=0, sort=False, ignore_index=True).values

del df_train, df_test

In [5]:
# start training of GaussianNB

from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import QuantileTransformer

start_tiem = time.time()

In [ ]:
_X = X[:len(Y)]
Xt = X[len(Y):]
X = _X

In [ ]:
clf = make_pipeline(QuantileTransformer(output_distribution='normal'), GaussianNB())

clf.fit( X, Y )

y_diff = Y - clf.predict_proba( X )[:,1]
yt_nb = clf.predict_proba( Xt )[:,1]

Y = y_diff

In [ ]:
# start training of LightGBM

import lightgbm as lgb
from sklearn.model_selection import KFold

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [ ]:
n_predict = 0
valid = np.zeros( (len(test_ID),) )
for fold_id, (IDX_train, IDX_test) in enumerate(KFold(n_splits=10, random_state=random_state, shuffle=False).split(Y)):
	X_train = X[IDX_train]
	X_test = X[IDX_test]
	Y_train = Y[IDX_train]
	Y_test = Y[IDX_test]

	lgb_params = {
		"objective" : "regression",
		"metric" : "mse",
		"max_depth" : 2,
		"num_leaves" : 2,
		"learning_rate" : 0.055,
		"bagging_fraction" : 0.3,
		"feature_fraction" : 0.15,
		"lambda_l1" : 5,
		"lambda_l2" : 5,
		"bagging_seed" : fold_id+random_state,
		"verbosity" : 1,
		"seed": fold_id+random_state
	}

	lgtrain = lgb.Dataset(X_train, label=Y_train)
	lgtest = lgb.Dataset(X_test, label=Y_test)
	evals_result = {}
	lgb_clf = lgb.train(lgb_params, lgtrain, 35000, 
						valid_sets=[lgtrain, lgtest], 
						early_stopping_rounds=500, 
						verbose_eval=2000, 
						evals_result=evals_result)
	valid += lgb_clf.predict( Xt ).reshape((-1,))
	n_predict += 1
	if time.time() - start_tiem > 6900:
		break

In [ ]:
valid = (valid / n_predict) + yt_nb
valid = np.clip(valid, 0.0, 1.0 )

print('save result.')
pd.DataFrame({'ID_code':test_ID,'target':valid}).to_csv('submission.csv',index=False)
print('done.')